# Imports and Functions

In [2]:
import pandas as pd
import numpy as np

# Goals

- for each pair of players (A,B), we define the fear score of A when facing B is the hit rate, such that B is closest defender when A is shooting. Based on the fear score, for each player, please find out who is his "most unwanted defender"
- for each player, we define the comfortable zone of shooting as a matrix of {SHOT_DIST, CLOSE_DEF_DIST, SHOT CLOCK}. Develop a MapReduce-based algorithm to classify each player's records into 4 comfortable zones. Considering the hit rate, which zone is the best for James Harden, Chris Paul, Stephen Curry, and Lebron James. 

# Notes